La **Ley de los Grandes Números** (LGN) es un resultado fundamental de la probabilidad que explica por qué, cuando repetimos un experimento aleatorio muchas veces, el promedio de los resultados tiende a acercarse al valor esperado verdadero de la variable.

---

## 1️⃣ Formulación básica

Si $X_1, X_2, X_3, \dots$ son **variables aleatorias independientes e idénticamente distribuidas (i.i.d.)** con media finita $\mu = E[X_i]$, entonces:

$$
\overline{X}_n = \frac{1}{n} \sum_{i=1}^n X_i \xrightarrow{n \to \infty} \mu
$$

Es decir, la media muestral $\overline{X}_n$ **converge** al valor esperado $\mu$ cuando el número de observaciones $n$ crece.

---

## 2️⃣ Dos versiones

* **Ley Débil**:
  Para cualquier $\varepsilon > 0$,

  $$
  P\left( \left| \overline{X}_n - \mu \right| > \varepsilon \right) \to 0 \quad \text{cuando } n \to \infty
  $$

  (La probabilidad de que la media muestral se aleje mucho de la media verdadera tiende a cero.)

* **Ley Fuerte**:

  $$
  P\left( \lim_{n \to \infty} \overline{X}_n = \mu \right) = 1
  $$

  (La media muestral **casi seguramente** converge a la media verdadera.)

---

## 3️⃣ Interpretación intuitiva

* Cuantas más repeticiones hagas, más se “compensan” las fluctuaciones aleatorias.
* El promedio de las observaciones se estabiliza alrededor del valor esperado.
* Esto es lo que garantiza que, por ejemplo, en métodos de **Monte Carlo** las simulaciones converjan al valor de la integral o la media que buscas.

---

## 4️⃣ Ejemplo

Si lanzas un dado justo:

* Valor esperado $E[X] = 3.5$.
* Si lanzas 10 veces, la media muestral podría ser 3.2, 3.9, etc.
* Si lanzas 1 millón de veces, la media estará muy cerca de 3.5.

---

Si quieres, puedo **simular en Python** la Ley de los Grandes Números para que veas cómo la media muestral de una Uniforme(0,1) se acerca a 0.5 al aumentar $n$.
¿Quieres que te lo muestre?


In [1]:
import matplotlib
matplotlib.use("TkAgg")  # o "Qt5Agg" si tienes PyQt5 instalado


In [2]:
# Interactivo: Ley de los Grandes Números con Uniforme(0,1)
# - Deslizador para elegir cuántas muestras mostrar (n)
# - Botón para regenerar una nueva simulación
#
# Reglas del entorno:
# * Usamos matplotlib (no seaborn)
# * Una sola figura por gráfico (sin subplots múltiples)
# * No especificamos colores manualmente

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

# Semilla variable (se vuelve a establecer al "Resamplear")
rng = np.random.default_rng()

# Parámetros
N_MAX = 50000
N_INIT = 2000

# Generamos una simulación inicial
samples = rng.random(N_MAX)  # Uniforme(0,1)
cum_mean = np.cumsum(samples) / np.arange(1, N_MAX + 1)

# Figura y ejes
fig, ax = plt.subplots(figsize=(8, 5))
plt.subplots_adjust(bottom=0.25)  # espacio para controles

# Líneas iniciales
(line_mean,) = ax.plot(np.arange(1, N_INIT + 1), cum_mean[:N_INIT], lw=1.5, label="Media muestral")
(line_true,) = ax.plot([1, N_INIT], [0.5, 0.5], linestyle="--", lw=1.2, label="Valor esperado (0.5)")

ax.set_title("Ley de los Grandes Números (Uniforme(0,1))")
ax.set_xlabel("Número de muestras (n)")
ax.set_ylabel("Media muestral de U")
ax.set_xlim(1, N_INIT)
ax.set_ylim(min(cum_mean[:N_INIT].min(), 0.0), max(cum_mean[:N_INIT].max(), 1.0))
ax.legend(loc="best")

# Controles: Slider para n
ax_n = plt.axes([0.15, 0.12, 0.7, 0.03])
slider_n = Slider(ax=ax_n, label="n", valmin=10, valmax=N_MAX, valinit=N_INIT, valstep=10)

# Botón para resincronizar (nueva simulación)
ax_btn = plt.axes([0.15, 0.05, 0.2, 0.05])
btn_resample = Button(ax_btn, "Resamplear")

def update_plot(n_val):
    n = int(n_val)
    # Actualizamos los datos mostrados
    line_mean.set_data(np.arange(1, n + 1), cum_mean[:n])
    line_true.set_data([1, n], [0.5, 0.5])
    # Reajustar límites para que sea informativo
    ax.set_xlim(1, n)
    local_min = min(cum_mean[:n].min(), 0.0)
    local_max = max(cum_mean[:n].max(), 1.0)
    # Damos un pequeño margen visual
    pad = 0.02 * (local_max - local_min + 1e-9)
    ax.set_ylim(local_min - pad, local_max + pad)
    fig.canvas.draw_idle()

def on_slider_change(val):
    update_plot(val)

def on_resample(event):
    global samples, cum_mean
    samples = rng.random(N_MAX)
    cum_mean = np.cumsum(samples) / np.arange(1, N_MAX + 1)
    update_plot(slider_n.val)

slider_n.on_changed(on_slider_change)
btn_resample.on_clicked(on_resample)

# Render inicial
update_plot(N_INIT)
plt.show()
